In [11]:
import re
import cv2 as cv
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Define Classes

In [12]:
class_names = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    "+",
    ".",
    "/",
    "=",
    "*",
    "-",
    "X",
    "Y",
    "Z",
]

# Load Model

In [10]:

model = tf.keras.models.load_model("models/19_class.h5")

OSError: No file or directory found at models/19_class.h5

# Obtain the eqn from image

In [13]:
def get_countours(image):
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    edged = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY_INV, 11, 4)
    (contours, _) = cv.findContours(edged, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    return contours

def get_bounding_box(contours):
    chars_bb = []
    for contour in contours:
                contour = contour.reshape((contour.shape[0],contour.shape[2]))
                left_tc = np.amin(contour, axis=0)
                right_bc = np.amax(contour, axis=0)
                min_x = left_tc[0]
                max_x = right_bc[0]
                min_y = left_tc[1]
                max_y = right_bc[1]
                chars_bb.append([min_x,min_y,max_x,max_y])
    return chars_bb

def replace_duplicate_chars(chars_bb):
    chars_bb.sort()
    for i, box in enumerate(chars_bb):
        try:
            next_box = chars_bb[i+1]
        except IndexError:
            break
   
        if abs(box[0] - next_box[0]) <= 30:
            min_x = min(box[0],next_box[0])
            min_y = min(box[1],next_box[1])
            max_x = max(box[2],next_box[2])
            max_y = max(box[3],next_box[3])
            new_box = [min_x,min_y,max_x,max_y]
            chars_bb[i] = new_box
            chars_bb.remove(next_box)
    return chars_bb
    
def get_crops(chars_bb, image):
    croped_images = []
    copy = image.copy()
    for box in chars_bb:
        x_min = box[0]
        y_min = box[1]
        height = abs(box[0]-box[2])
        width = abs(box[1]-box[3])
        character = copy[y_min:y_min+width , x_min:x_min+height]
        croped_images.append(character)
    return croped_images
        
def extra_padding(images, padding=50):
    padded_images = []
    for image in images:
        padded_images.append(cv.copyMakeBorder(image, top=padding, bottom=padding, left=padding, right=padding, borderType=cv.BORDER_CONSTANT, value=(255,255,255)))
    return padded_images

def resize_image(images):
    resized_images = []
    for img in images:
        resized_img = cv.resize(img, (100, 100), interpolation=cv.INTER_LINEAR)
        resized_images.append(resized_img)
    return resized_images

def get_prediction(images):
    predictions = []
    for image in images:
        image = np.expand_dims(image, axis=0)
        image = image.astype('float32')/255

        prediction = model.predict(image)
        label = class_names[np.argmax(prediction)]
        confidence = np.max(prediction)*100
        confidence = str(confidence)[:2]
        predictions.append((label, confidence))
    
    return predictions

def get_eqn(predictions):
    eqn = ""
    for prediction in predictions:
        eqn += prediction[0]
    return eqn


In [14]:
img_path = 'test_cases/lin_eqn.png'
image = cv.imread(img_path)

contours = get_countours(image)
char_bb = get_bounding_box(contours)
char_bb = replace_duplicate_chars(char_bb)
cropped_images  = get_crops(char_bb, image)
padded_images = extra_padding(cropped_images)
resized_images = resize_image(padded_images)
predictions = get_prediction(resized_images)
eqn = get_eqn(predictions)

eqn

error: OpenCV(4.5.4) /tmp/pip-req-build-kv0l0wqx/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


# Standardize the equation

In [15]:
def contains_nums(string):
    return any(i.isdigit() for i in string)


def standardize_eqn(equation):
    """
    Standardizes an equation by removing = and shift intercept to left.
    :param equation: equation to standardize eg 2x+3y=7
    :return: standardized equation eg 2x+3y-7
    """
    left, right = equation.split('=')[0], equation.split('=')[1]
    
    left_sub = re.sub("[+-]?\d+[XxYyZz]|[+-]?\d+\.\d+[XxYyZz]","", left)
    
    if left_sub == '': 
        intercept = -1* float(right)
        if intercept > 0.0: intercept = '+' + str(intercept)
        equation = left+ str(intercept)

    elif left_sub == left:
        intercept = -1* float(right)
        if intercept > 0.0: intercept = '+' + str(intercept)     
        equation = left+ str(intercept)
    
    elif contains_nums(left_sub):
        equation = left
    
    elif not contains_nums(left_sub):
        intercept = -1* float(right)
        if intercept > 0.0: intercept = '+' + str(intercept)      
        equation = left+ str(intercept)
        
    else:
        equation = left
        
    
    return equation

In [16]:
eqn_list = ["2x+3y=7", "x+y=7","3x+2y-7=0", "-7+3x-2y=0", "7+3x-2y=0","3x+5y+8=0", "x=3","x+y+z=0",'x+4y=2','2x-y=-2','x-y=-10']
for eqn in eqn_list:
    print(standardize_eqn(eqn))

2x+3y-7.0
x+y-7.0
3x+2y-7
-7+3x-2y
7+3x-2y
3x+5y+8
x-3.0
x+y+z-0.0
x+4y-2.0
2x-y+2.0
x-y+10.0


### This function takes standardized eqn and returns their coefficient and intercept

In [17]:
def get_coffecient_2d(equation):   
    """
    Returns the coefficients and intercept of the 2nd degree term in an equation.
    :param : eg: 2x+3y=7
    :return: eg: (2,3,7)
    """ 
    try:
        coef_x = re.findall('-?[0-9.]*[Xx]', equation)[0][:-1]
    except:
        coef_x = 0.0
    
    try:
        coef_y = re.findall('-?[0-9.]*[Yy]', equation)[0][:-1]
    except:
        coef_y = 0.0
    
    intercept = re.sub("[+-]?\d+[XxYy]|[+-]?\d+\.\d+[XxYy]","", equation)
    intercept = re.findall('[+-]+\d+', intercept)[0]

    if coef_x == '': coef_x =1.0
    elif coef_x == '-': coef_x=-1.0
    
    if coef_y == '': coef_y =1.0
    elif coef_y == '-': coef_y=-1.0
   

    return [float(coef_x), float(coef_y), float(intercept)]

In [18]:
eqn_list = ['2x+3y+1', 'x+y+1', 'x-3.0y-5', '-x+3y+7.0', 'x-3.0']
for eqn in eqn_list:
    print(get_coffecient_2d(eqn))

[2.0, 3.0, 1.0]
[1.0, 1.0, 1.0]
[1.0, -3.0, -5.0]
[-1.0, 3.0, 7.0]
[1.0, 0.0, -3.0]


In [19]:
def get_coffecient_3d(equation):   
    """
    Returns the coefficients and intercept of the 2nd degree term in an equation.
    :param : eg: 2x+3y=7
    :return: eg: (2,3,7)
    """ 
    try:
        coef_x = re.findall('-?[0-9.]*[Xx]', equation)[0][:-1]
    except:
        coef_x = 0.0
    
    try:
        coef_y = re.findall('-?[0-9.]*[Yy]', equation)[0][:-1]
    except:
        coef_y = 0.0
        
    try:
        coef_z = re.findall('-?[0-9.]*[Zz]', equation)[0][:-1]
    except:
        coef_z = 0.0
    
    intercept = re.sub("[+-]?\d+[XxYyZz]|[+-]?\d+\.\d+[XxYyZz]","", equation)
    intercept = re.findall('[+-]+\d+', intercept)[0]

    if coef_x == '': coef_x =1.0
    elif coef_x == '-': coef_x=-1.0
    
    if coef_y == '': coef_y =1.0
    elif coef_y == '-': coef_y=-1.0
    
    if coef_z == '': coef_z =1.0
    elif coef_z == '-': coef_z=-1.0
   

    return [float(coef_x), float(coef_y), float(coef_z),float(intercept)]

In [20]:
eqn_list = ['x+y+z+1','x+y+10','-x+y+3z-0.0']
for eqn in eqn_list:
    print(get_coffecient_3d(eqn))

[1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 0.0, 10.0]
[-1.0, 1.0, 3.0, -0.0]


# Try solving some eqn

In [21]:
def solve_2d(equtations):
    a = []
    b = []
    for i,eqn in enumerate(equtations):
        std_eqn = standardize_eqn(eqn)
        x,y, c = get_coffecient_2d(std_eqn)
        a.append([x,y])
        b.append(c)
    
    a,b = np.asanyarray(a), np.asanyarray(b)
    
    try:
        soln = np.linalg.solve(a, b)
    except:
        soln='No Solution'
    
    return soln
        
        

In [22]:
eqns = [
  '3x+4y=5',
'x-y+6=0'
]

solve_2d(eqns)

array([ 2.71428571, -3.28571429])

In [23]:
def solve_3d(equtations):
    a = []
    b = []
    for i,eqn in enumerate(equtations):
        std_eqn = standardize_eqn(eqn)
        x,y,z, c = get_coffecient_3d(std_eqn)
        a.append([x,y,z])
        b.append(c)
    
    a,b = np.asanyarray(a), np.asanyarray(b)
    try:
        soln = np.linalg.solve(a, b)
    except:
        soln='No Solution'    

    
    return soln
        
        

In [24]:
eqns = [
   'x+y=9',
   'x+5y=10',
   'x+y=11'
]

solve_3d(eqns)

'No Solution'